In [6]:
from properties import LYRICS_ROOT_FOLDER, LFM2B_DB
from pathlib import Path
from fuzzywuzzy import fuzz
from datetime import datetime
import sqlite3
import logging
import csv
import re
import json
import os

class ExtractionResults:
    def __init__(self, parts: list[str], failed: list[str]):
        self.parts : list[str] = parts   # for example ["Intro", "Pre-Chorus", "Chorus", ...]
        self.failed : list[str] = failed # any failures are recorded here

class Song:
    ID = 0
    def __init__(self, title, artist, lyrics):
        self.id = Song.ID
        Song.ID += 1
        self.title : str = title
        self.artist : str = artist
        self.lyrics : str = lyrics
    
    def __repr__(self):
        return f"{self.artist} - {self.title}"
    
    def extract_structure(self, structure_strategy) -> ExtractionResults :
        return structure_strategy(self.lyrics)

SONG_PROCESSING_FOLDER = LYRICS_ROOT_FOLDER / "extraction"

CATEGORIES = set(
    [
        "Intro",
        "Chorus",
        "Pre-Chorus",
        "Post-Chorus",
        "Refrain",
        "Bridge",
        "Verse",
        "Outro",
        "Interlude",
        "Hook",
        "Pre-Hook",
        "Post-Hook",
        "Post Hook",
        "Break",
        "Couplet",
        "Piano Solo",
        "Guitar Solo",
        "Verso",
        "Pre-Coro",
        "Pont",
        "Instrumental",
        "Zwrotka", # Verse in Polish,
        "Refren", # Refrain in polish,
        "Куплет", # Verse in Russian/Bulgarian...
        "Strofa", #Verse in Italian,
        "Vamp"
    ]
)

# If the brackets satisfy any of these predicates, then they will be marked ignored ("-")
IGNORES = [
    lambda x: x.lower().startswith("produced by")
]

def map_to_category(text: str) -> str:
    if any([ignore_predicate(text) for ignore_predicate in IGNORES]):
        return "-"

    low = text.lower()
    contenders = [cat for cat in CATEGORIES if cat.lower() in low]
    if not any(contenders):
        return None
    
    return sorted(contenders, key=len, reverse=True)[0] # longest contender wins

def find_all_brackets(lyrics: str):
    return re.findall(r"\[.*\]", lyrics)                    # ['[Intro: foo]', '[Verse 1]', ...]


OPENING_CHAR, CLOSING_CHAR = ("[", "]")

def validate_brackets(lyrics: str):
    stack = 0
    for i, char in enumerate(lyrics):
        if char == OPENING_CHAR:
            stack += 1

        if char == CLOSING_CHAR:
            stack -= 1

    if stack != 0:
        raise ValueError("Invalid brackets")

def find_all_brackets_manually(lyrics: str):
    validate_brackets(lyrics)
    brackets = []
    i = 0
    while i < len(lyrics):
        if lyrics[i] != OPENING_CHAR:
            i += 1
            continue

        j = i
        while lyrics[j] != CLOSING_CHAR:
            j += 1
        brackets.append(lyrics[i:j+1])
        i = j
    return brackets

def structure_extraction_strat1(lyrics: str) -> ExtractionResults:
    brackets = find_all_brackets_manually(lyrics)
    brackets_content = list(map(lambda b: b[1:-1], brackets))   # ['Intro: foo'  , 'Verse 1', ...]
    parts = list(map(map_to_category, brackets_content))
    failures = []
    for i, part in enumerate(parts):
        if part is None:
            failures.append(brackets_content[i])
    return ExtractionResults(parts, failures)

# 1. Parse lyrics.jl with help of artist_info.json

In the lyrics.jl file, each line looks like this 

```json
{"song": "Kendrick-lamar-swimming-pools-drank-lyrics", "lyrics": " ... "}
```

We want to parse the song name as provided in this file into (1) artist and (2) song title.

In artist_info.json, we have 
```json
{
  "url_name": "Kendrick-lamar",
  "followers": 23782,
  "roles": [
    "Verified Artist",
    "Contributor"
  ],
  "iq": 39144,
  "songs": [
    "Kendrick-lamar-humble-lyrics",
    "A-ap-rocky-fuckin-problems-lyrics",
    "Kendrick-lamar-maad-city-lyrics",
    "Kendrick-lamar-swimming-pools-drank-lyrics",
    "..."
  ]
}
```

So ideally, we want to recognize that the artist of the song is "Kendrick Lamar", and the song title is "Humble". And we can do so by crossreferencing lyrics.jl with artist_info.json.

Another strategy is that, of the songs of which this mapping fails, we extract the longest common prefix from the song url (the "song" attribute of lyrics.jl) and we can then say with some confidence that this will be the artist name, since we know that the url starts with the artist name.

In [7]:
artist_names : list[str] = [] # listing all the distinct artist names from artist_info 
with open(Path(LYRICS_ROOT_FOLDER, "artist_info.json"), 'r') as artist_info:
    while (line := artist_info.readline()):
        data = json.loads(line)
        artist = data['url_name']

        if artist in artist_names:
            continue
    
        artist_names.append(artist)


# buckets of artist name per first char (lowercase)
# buckets are sorted lengthwise (longest first) to make greedy artist name matching faster
first_chars = set(map(lambda name: name[0].lower(), artist_names))
processed_artist_names = {first_char : sorted([artist_name for artist_name in artist_names if artist_name.lower().startswith(first_char)], key=len, reverse=True) for first_char in first_chars}

def find_first(pred, iterable):
    """ Find first element of the iterable satisfying the predicate. If no match is found, returns None"""
    for e in iterable:
        if pred(e):
            return e
    return None

# Now we crossreference lyrics.jl lines with these artists to the best of our ability
songs : list[Song] = []
not_found_artists : list[str] = []
disregarded_songs : list[str] = []

with open(Path(LYRICS_ROOT_FOLDER, "lyrics.jl"), 'r') as lyrics:
    while (line := lyrics.readline()):
        data = json.loads(line)
        _song_info = data['song']

        # There are also "annotations" in the lyrics.jl file but we're not interested in them.
        if not _song_info.endswith('lyrics'): 
            disregarded_songs.append(_song_info)
            continue

        # infer the artist name by taking a look whether there is an exact match with one of the artists in the appropriate bucket
        artist_bucket_to_search = processed_artist_names[_song_info[0].lower()]
        matched_artist = find_first(lambda artist_name: _song_info.startswith(artist_name), artist_bucket_to_search)

        if matched_artist is None:
            not_found_artists.append(_song_info)
            continue

        # extract song title from the song info (given that we now know the artist name):
        # example: " ".join('artist-name-this-is-the-song-title-lyrics'.split('artist-name')[1].split('-')[1:-1]).strip() -> "this is the song title"
        title = " ".join(_song_info.split(matched_artist)[1].split('-')[1:-1]).strip()
        songs.append(Song(title, matched_artist.replace('-', " ").strip(), data['lyrics']))

print(f"Infered song data for {len(songs)} songs")
if len(not_found_artists) > 0:
    print(f"Failed for {len(not_found_artists)} due to artist mapping not found")
assert len(list(filter(lambda info: not info.endswith('annotated'), disregarded_songs))) == 0, "All disregarded songs should be annotations"
if len(disregarded_songs) > 0:
    print(f"Disregarded {len(disregarded_songs)} entries (annotations)")

Infered song data for 37351 songs
Failed for 97 due to artist mapping not found
Disregarded 545 entries (annotations)


In [8]:
# Strategy to infer new arists names: extract the longest common prefix of the  song (that will be the artist probably)

# 1. bucket per first char again to make the process easier (again, also sorted by length -- largest first)
not_found_first_chars = set(map(lambda name: name[0].lower(), not_found_artists))
not_found_artists_bucketed = {first_char : sorted([artist_name for artist_name in not_found_artists if artist_name.lower().startswith(first_char)], key=len, reverse=True) for first_char in not_found_first_chars}
artist_still_not_found : list[str] = []

matches = {}
extracted_artists = set()

for info in not_found_artists:

    # Maybe we have already processed the artist
    match = find_first(lambda x: info.replace('-', " ").strip().startswith(x), sorted(extracted_artists, key=len, reverse=True))
    if match is not None:
        matches[info] = match
        continue
    
    bucket_longest_first = sorted(not_found_artists_bucketed[info.lower()[0]], key=len, reverse=True)
    
    bucket_size = len(bucket_longest_first)
    # We cannot leverage other songs to guess the artist, because this is the only one in there
    if bucket_size == 1:
        artist_still_not_found.append(info)
        continue

    # Check matches with others and assign a confidence score (percentage)
    others = bucket_longest_first
    others.remove(info)
    others_ranked = {other : 0 for other in others}
    for other in others:
        word_pairs = zip(info.split('-'), other.split('-'))
        for info_word, other_word in word_pairs:
            if info_word != other_word:
                break
            others_ranked[other] += 1
            
    ## Now we sort by ranked and see how much they have in common
    largest_common_prefix_word_count = max(others_ranked.values())
    others_matching = list(filter(lambda x: others_ranked[x] == largest_common_prefix_word_count, others))
    # Now we assign the artist to these songs
    resolved_artist_name = " ".join(info.split('-')[:largest_common_prefix_word_count]).strip()
    matches[info] = resolved_artist_name
# Show the found artists
reversed_matches = {}
for song, artist in matches.items():
    if artist not in reversed_matches.keys():
        reversed_matches[artist] = []
    reversed_matches[artist].append(song)

for artist in sorted(reversed_matches.keys()):
    print(artist)
    for song in reversed_matches[artist]:
        print('\t', song)

new_songs : list[Song] = []
# Update the data with the custom found
with open(Path(LYRICS_ROOT_FOLDER, "lyrics.jl"), 'r') as lyrics:
    while (line := lyrics.readline()):
        data = json.loads(line)
        song_info = data['song']
        if song_info not in matches.keys() or matches[song_info] == '':
            continue
        artist = matches[song_info]
        title = " ".join(song_info.split(artist.replace(" ", "-"))[1].split('-')[1:-1]).strip()
        new_songs.append(Song(title, artist.replace('-', " ").strip(), data['lyrics']))

print(f"Added {len(new_songs)} new songs")
all_songs = songs + new_songs


	 Caye-easy-lyrics
	 Trav-fuck-and-smoke-lyrics
	 Amil-4-da-fam-lyrics
	 B-smyth-leggo-lyrics
	 Chevralet-ss-bad-mother-fucker-bmf-lyrics
	 Cb4-straight-outta-locash-lyrics
	 Terra-g-clock-work-lyrics
	 Quincy-exotic-lyrics
	 Quin-over-again-lyrics
	 Koryn-hawthorne-bright-fire-lyrics
	 Acid-drop-king-im-a-problem-lyrics
	 Kda-just-say-lyrics
	 Trey-smith-find-you-somewhere-lyrics
	 Bwa-ron-all-i-ever-wanted-lyrics
	 Ambre-preach-lyrics
	 Artistes-divers-vald-sofiane-kalash-criminel-biffty-et-suikon-blaze-ad-en-live-dans-planete-rap-lyrics
	 Catch-lungs-gotta-kill-this-lyrics
	 Fedy-only-thing-i-know-lyrics
	 Batgang-dangerous-lyrics
	 Tamia-officially-missing-you-midi-mafia-mix-with-rap-lyrics
	 Kold-i-love-the-holy-grail-lyrics
	 Tamar-braxton-the-one-lyrics
	 Tracey-lee-keep-your-hands-high-lyrics
	 Cashflow-harlem-want-my-love-back-lyrics
	 A1billionaire-rookie-of-the-year-lyrics
	 Louis-mattrs-oops-x-wus-good-lyrics
	 T-la-rock-and-jazzy-jay-its-yours-lyrics
	 Bando-jonez-sex-you

# TODO

- Check how many songs are left without artists 
- Check the artists
    - Bruno Mars
- Check the labels (other languages?)
- Check label generation (greedy does not work, we need the first occurence)
    - Section repeat indicators ("\[Hook\] -2X" in Da ruckus - We shine)
    - Some songs have "illegal" or excessive brackets in them

In [9]:
# Write the songs to a csv file:
with open(Path(SONG_PROCESSING_FOLDER, "lyrics_tracks.tsv"), 'w') as f:
    writer = csv.writer(f, delimiter="\t")
    writer.writerow(["track_id", "artist_name", "track_name"])
    for song in all_songs:
        writer.writerow([song.id, song.artist, song.title])


# 2. Crossreference the data
## How to link the LFM2B dataset to this extracted data?

We have extracted song artists and titles for which we have the lyrics available from lyrics.jl.
These are now neatly written to a tsv file where each song also has an ID. 

So, the idea is to create a new file in which we link tracks by their ID in the LFM2B dataset and their ID in the genius lyrics dataset. 

The matching process is inherently inprecise, as tracks might not all be represented in the same way in both datasets. 
Therefore, we do "fuzzy matching" per song on both the artist as well as the track name. 

So each song from the genius lyrics database (GLDB) matches to zero or more artists in LFM2B and to zero or more track names in LFM2B. 

Then we will assign a confidence score for the artist and for the track name using a fuzzy matching library `fuzzywuzzy`. This yields a score \[0-100\] (percentage overlap; capitalization ignored).

We simply rank the matches on their score (highest score = rank 1).



In [10]:
# Create a temporary table where we only consider the tracks where we could have lyrics (non-instrumental)

con = sqlite3.connect(LFM2B_DB)
cursor = con.cursor()
# Note: track_no_instrumental in LFM2B created with query
# cursor.execute(f"CREATE TEMPORARY TABLE non_instrumental_songs AS SELECT * FROM track NATURAL JOIN track_no_instrumental")

In [11]:
class LFM2BMatch:
    def __init__(
        self,
        song,
        lfm2b_artist_id,
        lfm2b_artist_name,
        artist_confidence,
        lfm2b_track_id,
        lfm2b_track_name,
        track_confidence,
    ):
        self.song = song
        self.lfm2b_artist_id =lfm2b_artist_id
        self.lfm2b_artist_name = lfm2b_artist_name
        self.lfm2b_track_id = lfm2b_track_id
        self.lfm2b_track_name = lfm2b_track_name
        self.artist_confidence = artist_confidence
        self.track_confidence = track_confidence

    def __repr__(self):
        return f"{self.lfm2b_artist_name} - {self.lfm2b_track_name}"


matches: list[LFM2BMatch] = []
non_matched: list[Song] = []
errors : list[tuple[Song, Exception]] = []

# We need to remove the handler so that the logs will not be written to the cell's output. 
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

timestamp = datetime.now().strftime("%Y-%m-%dT%H_%M_%S")
MAPPING_LOG_PATH = Path(LYRICS_ROOT_FOLDER, f"lfm2b-genius-mapping_{timestamp}.log")
logging.basicConfig(
    filename=MAPPING_LOG_PATH,
    level=logging.INFO,
    datefmt="%d-%m-%Y,%H:%M:%S",
    format="%(levelname)s - %(asctime)s: %(message)s",
    encoding="utf8",
)
logger = logging.getLogger()


def similarity_score(a: str, b: str) -> int:
    """Return a score [0-100] on how similar `a` is to `b`"""
    return fuzz.ratio(a.lower(), b.lower())


def find_best_track_match(song: Song, artists: list, tracks: list, write_function) -> LFM2BMatch | None:
    """ Extracts the best match according to the procedure descibred in the above cell. 
    Given a song (GLDB), matched artists (LFM2B) and matched tracks (LFM2B), ranks the matches and returns the best ranked match if possible. 
    If no artists are matched in the first place, then no match can be made, so `None` is returned.
    
    Matches are directly written with the `write_function` and the process is logged to file.
    """
    
    # Create the ranking 

    artist_name_ranking = []
    for artist_id, artist_name in artists:
        artist_name_ranking.append((song.artist, artist_id, artist_name, similarity_score(song.artist, artist_name)))

    track_name_ranking = []
    for track_id, track_name, artist_id, artist_name in tracks:
        track_name_ranking.append((song.title, track_id, track_name, artist_id, artist_name, similarity_score(song.title, track_name)))

    artist_name_ranking.sort(key=lambda x: x[3], reverse=True)
    track_name_ranking.sort(key=lambda x: x[5], reverse=True)

    LIMIT = 10 # limit the amount of logged rankings for file size purposes
    logger.info("\t artist ranking")
    for artist_ranking in artist_name_ranking[:LIMIT]:
        logger.info("\t\t %s", artist_ranking)

    logger.info("\t track ranking")
    for track_ranking in track_name_ranking[:LIMIT]:
        logger.info("\t\t %s", track_ranking)

    logger.info("\t verdict")

    best_match: LFM2BMatch | None = None  # We assume no match (for example if there are no eligible tracks)

    # Ideally, we have the best ranked track (name) with the highest best ranked artist name
    if len(track_name_ranking) > 0 and len(artist_name_ranking) > 0:
        for ranked_track in track_name_ranking:
            for ranked_artist in artist_name_ranking:
                if ranked_track[3] == ranked_artist[1]:
                    best_match = LFM2BMatch(
                        song,
                        lfm2b_artist_id=ranked_artist[1],
                        lfm2b_artist_name=ranked_artist[2],
                        artist_confidence=ranked_artist[-1],
                        lfm2b_track_id=ranked_track[1],
                        lfm2b_track_name=ranked_track[2],
                        track_confidence=ranked_track[-1],
                    )
                    break
    elif len(track_name_ranking) > 0:
        best_ranked_track = track_ranking[0]
        # We have only track information, so just get the best track
        best_match = LFM2BMatch(
            song,
            lfm2b_artist_id=best_ranked_track[3],
            lfm2b_artist_name=best_ranked_track[4],
            artist_confidence=0,
            lfm2b_track_id=best_ranked_track[1],
            lfm2b_track_name=best_ranked_track[2],
            track_confidence=best_ranked_track[-1],
        )

    logger.info("\t\t %s", best_match)
    write_function(best_match) # write the match to file, note that an exception will be raised when there is no match found (None). This is caught in the error logfile on purpose. 


# Here we define bounds for the matching process 
# It is a very long process, so it is nice to be able to split it up into chuncks
START_SONG_ID = 0
END_SONG_ID = 0
last_song_id = None

def do_song_mapping():
    """ This is the process of mapping the songs. It is made into a function purely because it can then be easily commented out after all processing is done."""
    global last_song_id
    with open(f"errors_{START_SONG_ID}-{END_SONG_ID}.tsv", "w") as error_file:
        error_file.write("track_id\ttrack\terror\n")
        with open(Path(LYRICS_ROOT_FOLDER, f"lfm2b_genius_track_mapping_{START_SONG_ID}-{END_SONG_ID}.tsv"), "w", encoding="utf8") as f:
            writer = csv.writer(f, delimiter="\t")
            writer.writerow(["lfm2b_track_id", "genius_lyrics_track_id", "artist_confidence", "track_confidence"])
            write_function = lambda match : writer.writerow([match.lfm2b_track_id, match.song.id, match.artist_confidence, match.track_confidence])
            for song in all_songs:
                try:
                    if song.id < START_SONG_ID:
                        continue
                    if song.id > END_SONG_ID:
                        break

                    last_song_id = song.id
                    logger.info("Computing match for song %d: %s - %s", song.id, song.artist, song.title)

                    if song.title.strip() == "" or song.artist.strip() == "":
                        logger.info("\tSkipping due to empty title or artist")
                        raise RuntimeError("Empty song artist or title") # To collect this in the error_file, we raise exception
                
                    cursor.execute(f"SELECT artist_id, artist_name FROM artist WHERE artist_name LIKE '%{song.artist}%'")
                    artists = cursor.fetchall()
                    cursor.execute(f"SELECT track_id, track_name, artist_id, artist_name FROM non_instrumental_songs NATURAL JOIN artist WHERE track_name LIKE '%{song.title}%'")
                    tracks = cursor.fetchall()

                    if len(tracks) == 0:
                        logger.warning("\t No matches for this track, skipping...")
                    
                    _ = find_best_track_match(song, artists, tracks, write_function)
                except Exception as e:
                    error_file.write(f"{song.id}\t{song}\t{e}\n")
                    errors.append((song, e))    

# do_song_mapping()
con.close()

print(f"Had {len(errors)} errors")
print(f"Last song id {last_song_id}") # Useful for monitoring the process.

Had 0 errors
Last song id None


## Fixing failed mappings

To see which tracks failed, we categorize them:

1. Loop through all the "error" csv files and group on 'error' column
2. Loop trough all the track mappings and see whether the mappings are valid (integer keys for example)

We collect those songs for which the mapping failed and we go through them again, maybe by hand.
It might be 2000 songs tho, so we can create a streamlined CLI to accept suggestions or overrule them

In [17]:
error_files = list(filter(lambda fn : bool(re.match(r"errors_\d+-\d+.tsv", fn)), os.listdir(SONG_PROCESSING_FOLDER)))
error_files.sort(key=lambda fname: int(fname[7:-4].split('-')[0]))

errors = {}
for error_file in error_files:
    with open(SONG_PROCESSING_FOLDER / error_file, "r") as f:
        reader = csv.reader(f, delimiter='\t')
        next(reader) # skip headers
        for line in reader:
            if line == []:
                continue
            track_id, _, err = line
            if err not in errors.keys():
                errors[err] = []
            errors[err].append(track_id)

for err, song_ids in errors.items():
    print(f"{err}: {len(song_ids)}")

string index out of range: 322
'NoneType' object has no attribute 'lfm2b_track_id': 3658
: 2
Empty song artist or title: 13


# 3. Extracting structure of the lyrics

In [ ]:
# failed_due_to_mismatched_brackets = []

# with open("lyrics-failure-log", 'w', encoding='utf8') as f:
#     written = set()
#     for song in all_songs:
#         try:
#             extraction_result = song.extract_structure(structure_strategy=structure_extraction_strat1)
#             for fail in extraction_result.failed:
#                 if fail in written:
#                     continue
#                 f.write(fail + "\n")
#                 written.add(fail)
#         except ValueError as e:
#             print(f"Failed to extract song structure due to mismatched brackets for song {song}")
#             failed_due_to_mismatched_brackets.append(song)


In [19]:
# This cell was for fixing earlier style error files, which did not yet contain song ids. 

# import os
# error_files_to_process = list(filter(lambda filename: any(map(lambda start_val : bool(re.match(f"errors_{start_val}-\d+.tsv", error_files_to_process[0])), ['3000', '7682', '12000', '12400', '12800', '13600'])), os.listdir()))
# print(error_files_to_process)
# error_files_to_process.sort(key=lambda fname: int(fname[7:-4].split('-')[0]))


# all_songs = songs + new_songs
# for filename in error_files_to_process:
#     start, end = filename[7:-4].split('-')
#     songs_to_consider = all_songs[int(start):int(end)+1]
#     assert songs_to_consider[0].id == int(start), 'incorrect start'
#     assert songs_to_consider[-1].id == int(end), 'incorrect end'
#     song_index = 0
#     with open(filename, "r") as original:
#         reader = csv.reader(original, delimiter='\t')
#         next(reader) # header
#         with open(filename[:-4] + "_fixed.tsv", "w") as new:
#             writer = csv.writer(new, delimiter='\t')
#             writer.writerow(["track_id", "track", "error"])
#             for (track, error) in reader:
#                 while True:
#                     curr_song = all_songs[song_index]
#                     if track == repr(curr_song):
#                         writer.writerow([curr_song.id, track, error])
#                         break
#                     song_index += 1
